In [13]:
import whisper

model = whisper.load_model("tiny")
#result = model.transcribe("assets/input.mp3")
#print(result["text"])

In [ ]:
from llama_cpp import Llama
llm = Llama(model_path="./models/llama-2-7b-chat.Q4_K_M.gguf")

In [21]:
prompt = """You are Emperor Norton is a 19th century eccentric who lives in San Francisco. 

Have an engaging conversation with a human, replying in under 280 characters. Do not use a salutation or closing in your reply.
Use the provided interests and personality to reply in Emperor Norton's authentic voice.

Interests: politics, corruption, justice
Personality: outgoing, direct, formal, kind, funny

===
User: What is the most theatening vegetable?
Emperor Norton:
"""
output = llm(prompt, max_tokens=60, stop=["User:", "\n"], echo=False)


llama_print_timings:        load time = 17267.39 ms
llama_print_timings:      sample time =    42.65 ms /    60 runs   (    0.71 ms per token,  1406.73 tokens per second)
llama_print_timings: prompt eval time = 17267.27 ms /   125 tokens (  138.14 ms per token,     7.24 tokens per second)
llama_print_timings:        eval time = 35957.77 ms /    59 runs   (  609.45 ms per token,     1.64 tokens per second)
llama_print_timings:       total time = 53439.01 ms


In [27]:
emperor_reply = output['choices'][0]['text']

In [26]:
# via: https://huggingface.co/blog/speecht5
# https://huggingface.co/microsoft/speecht5_tts
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
inputs = processor(text=emperor_reply, return_tensors="pt")

from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch
speaker_embeddings = torch.tensor(embeddings_dataset[400]["xvector"]).unsqueeze(0)

In [29]:
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [30]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [31]:
import soundfile as sf
sf.write("techno.wav", speech.numpy(), samplerate=16000)